<img src="pic/icon_exercise_header_p3.png" width="100%"> <br>

# ICON Community Interface ComIn - Practical Exercise Notebooks

# <br> Exercise P3: Implementing a Diagnostic Function as a ComIn Plugin

---
* **Exercise P3**: In this exercise, we implement the calculation of two new diagnostic fields (2D variables) `RHI_MAX` and `QI_MAX`. We will also learn how to write a callback function which plots the new diagnostic fields.
* This exercise builds on what we have learned so far in <a href="P1_exercise.ipynb"><code>P1_exercise</code></a> and <a href="P2_exercise.ipynb"><code>P2_exercise</code></a>. To proceed, make sure that you have the binary `icon` configured with the `--enable-comin` option and the ComIn Python adapter obtained in the **previous exercises**.

<p style="height:2em;"/>

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> 
    Introduce two 2D diagnostic fields <code>RHI_MAX</code> and <code>QI_MAX</code> and set the metadata such as <i>unit</i> and <i>long_name</i> in your plugin <a href="scripts/comin_plugin.py"><code><code style="color:#2d4b9b">scripts/comin_plugin.py</code></a>.
    <p/>
    Compute them on prognostic cells in your callback function to represent their physical meanings:
<ul>
<li> <code>RHI_MAX</code>: Maximum relative humidity over ice (maximum in the vertical column, [%]) </li>
<li> <code>QI_MAX</code> : Maximum cloud ice content (maximum in the vertical column, [kg/kg]) </li>
</ul>
</div>

**Hints**: 
* You should access the following variables from ICON: `temp`, `qv`, `exner`, `qi`.
* Calculation of *relative humidity over ice (`RHI`)*: Insert the following helper function into your plugin script  
  This is a Python implementation of the Fortran version in the module <a href="https://gitlab.dkrz.de/icon/icon-model/-/blob/release-2024.07-public/src/atm_phy_nwp/mo_util_phys.f90"><code>src/atm_phy_nwp/mo_util_phys.f90</code></a> in ICON.

```python
def rhi(temp, qv, p_ex):
    import numpy as np
    rdv  = (rd := 287.04)/(rv := 461.51)
    pres = (p0ref := 100000) * np.exp(((cpd := 1004.64) / rd) * np.ma.log(p_ex))
    e_s  = 610.78 * np.ma.exp(21.875 * (temp - 273.15) / (temp - 7.66))
    e    = pres * qv / (rdv + (1.0-(rd/rv)) * qv)
    return 100.0 * e / e_s
```

* This is how you can define a mask for prognostic cells, similar to the previous exercise:
  
  ```python
    # mask for prognostic cells like step P2!!!
    mask_2d = (decomp_domain_np != 0)
    # define a 3d mask
    mask_3d = np.repeat(mask_2d[:,None,:], domain.nlev, axis=1)
    # only prognostic cells
    exner_np = np.ma.masked_array(np.squeeze(exner), mask = mask_3d)
    ```

### Solution:

```python
""" 
Calculation of 2D diagnostic fields `RHI_MAX` and `QI_MAX`.
This is a ComIn Python plugin designed for use in the ICON 2024 training course.
"""
import comin
import numpy as np

# domain_id
jg = 1
#domain descriptive data structure
domain = comin.descrdata_get_domain(jg)
decomp_domain_np = np.asarray(domain.cells.decomp_domain)

# first variable                                                           
var_descriptor = ("RHI_MAX", jg)
comin.var_request_add(var_descriptor, lmodexclusive=False)
comin.metadata_set(var_descriptor, zaxis_id = comin.COMIN_ZAXIS_2D,
                                   long_name = 'Maximum relative humidity over ice',
                                   units = '%')

# second variable
var_descriptor = ("QI_MAX", jg)
comin.var_request_add(var_descriptor, lmodexclusive=False)
comin.metadata_set(var_descriptor, zaxis_id = comin.COMIN_ZAXIS_2D,
                                   long_name = 'Maximum cloud ice content',
                                   units = 'kgkg')


def rhi(temp, qv, p_ex):
    rdv  = (rd := 287.04)/(rv := 461.51)
    pres = (p0ref := 100000) * np.exp(((cpd := 1004.64) / rd) * np.ma.log(p_ex))
    e_s  = 610.78 * np.ma.exp(21.875 * (temp - 273.15) / (temp - 7.66))
    e    = pres * qv / (rdv + (1.0-(rd/rv)) * qv)
    return 100.0 * e / e_s


@comin.register_callback(comin.EP_SECONDARY_CONSTRUCTOR)
def simple_python_constructor():
    # access the variables 
    global RHI_MAX, QI_MAX, temp, qv, qi, exner
    RHI_MAX = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("RHI_MAX", jg), flag=comin.COMIN_FLAG_WRITE)
    QI_MAX  = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("QI_MAX", jg),  flag=comin.COMIN_FLAG_WRITE)
    temp    = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("temp", jg))
    qv      = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("qv", jg))
    qi      = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("qi", jg))
    exner   = comin.var_get([comin.EP_ATM_WRITE_OUTPUT_BEFORE], ("exner", jg))


@comin.register_callback(comin.EP_ATM_WRITE_OUTPUT_BEFORE)
def simple_python_callbackfct():
    # mask for prognostic cells like step P2!!!
    mask_2d = (decomp_domain_np != 0)
    # define a 3d mask
    mask_3d = np.repeat(mask_2d[:,None,:], domain.nlev, axis=1)
    # only prognostic cells
    exner_np = np.ma.masked_array(np.squeeze(exner), mask = mask_3d)
    temp_np  = np.ma.masked_array(np.squeeze(temp),  mask = mask_3d)
    qv_np    = np.ma.masked_array(np.squeeze(qv),    mask = mask_3d)
    qi_np    = np.ma.masked_array(np.squeeze(qi),    mask = mask_3d)
    # compute RHI_MAX and QI_MAX in atmospheric column
    RHI_MAX_np = np.squeeze(RHI_MAX)
    QI_MAX_np  = np.squeeze(QI_MAX)
    RHI_MAX_np[:, :] = np.max(rhi(temp_np, qv_np, exner_np), axis = 1)
    QI_MAX_np[:, :]  = np.max(qi_np, axis = 1)
```

<p style="height:2em;"/>

### Run the ICON model

* Now that you have finished writing your plugin, the next step is to run the ICON model using the script <a href="scripts/icon_exercise_comin_run.ipynb"><code><code style="color:#2d4b9b">icon_exercise_comin_run.ipynb<code></a>  
* Modify your namelist to have the computed new variables (`RHI_MAX` , `QI_MAX`) in the model output.  
* At this point, as a **post-processing** step, you can use <a href="scripts/exercise_comin_plot_P3.ipynb"><code style="color:#2d4b9b">exercise_comin_plot_P3.ipynb</code></a> to visualize the `RHI_MAX` and `QI_MAX` fields from your outputs file. The obtained plot should look like <a href="reference/P3_comin_plugin/P3_post_processing.png"> <code><code style="color:#2d4b9b">P3_post_processing.png</code></a>.

<p style="height:2em;"/>

### Creating a Plot with ComIn

Of course, ComIn plugins that are implemented in Python can access the publicly available, vast library of useful Python modules.

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Exercise:</b> In the final exercise, add a callback function to your plugin that <b>uses the <code>matplotlib</code> and <code>cartopy</code> modules to create a plot</b> of <code>RHI_MAX</code> and <code>QI_MAX</code> at the last time step.
    </div>

**Hints:**
* To write a plotting function, you can refer to the example script in Section 10.3.1 of the <a href="https://www.dwd.de/DE/leistungen/nwv_icon_tutorial/nwv_icon_tutorial.html">ICON tutorial</a> to get some ideas.
* Think about which entry point is suitable to use for your callback function!
* Before plotting, it is essential to <b>gather the local arrays from each MPI processor into a single global array</b> on MPI rank 0. Subsequently, the plotting should happen on MPI rank 0 only.
  You can achieve this task using the helper function provided below. It can simply be used on your callback function.
```python
from mpi4py import MPI

comm   = MPI.Comm.f2py(comin.parallel_get_host_mpi_comm())
rank   = comm.Get_rank()


def util_gather(data_array : np.ndarray, root = 0):

    # 0-shifted global indices for all local cells (including halo cells):
    global_idx = np.asarray(domain.cells.glb_index) -1

    # no. of local cells (incl. halos):
    nc = domain.cells.ncells

    # to remove empty cells
    data_array_1d = data_array.ravel('F')[0:nc]
    decomp_domain_np_1d = decomp_domain_np.ravel('F')[0:nc]
    halo_mask = (decomp_domain_np_1d == 0)

    # to remove halo cells
    data_array_1d = data_array_1d [halo_mask]
    global_idx = global_idx[halo_mask]

    # gather operation
    data_buf = comm.gather((data_array_1d, global_idx), root=root)

    # reorder received data according to global_idx
    if rank == root:
        nglobal = sum([len(gi) for _, gi in data_buf])
        global_array = np.zeros(nglobal, dtype=np.float64)
        for data_array_i, global_idx_i  in data_buf:
            global_array[global_idx_i] = data_array_i
        return global_array
    else:
        return None

```

### Solution:

``` python

from mpi4py import MPI

comm   = MPI.Comm.f2py(comin.parallel_get_host_mpi_comm())
rank   = comm.Get_rank()

def util_gather(data_array : np.ndarray, root = 0):

    # 0-shifted global indices for all local cells (including halo cells):
    global_idx = np.asarray(domain.cells.glb_index) -1

    # no. of local cells (incl. halos):
    nc = domain.cells.ncells

    # to remove empty cells
    data_array_1d = data_array.ravel('F')[0:nc]
    decomp_domain_np_1d = decomp_domain_np.ravel('F')[0:nc]
    halo_mask = (decomp_domain_np_1d == 0)

    # to remove halo cells
    data_array_1d = data_array_1d [halo_mask]
    global_idx = global_idx[halo_mask]

    # gather operation
    data_buf = comm.gather((data_array_1d, global_idx), root=root)

    # reorder received data according to global_idx
    if rank == root:
        nglobal = sum([len(gi) for _, gi in data_buf])
        global_array = np.zeros(nglobal, dtype=np.float64)
        for data_array_i, global_idx_i  in data_buf:
            global_array[global_idx_i] = data_array_i
        return global_array
    else:
        return None


@comin.register_callback(comin.EP_ATM_TIMELOOP_AFTER)
def simple_plotfct():
    import cartopy
    from matplotlib import pyplot as plt
    import getpass
    
    user = getpass.getuser()

    cx = np.rad2deg(domain.cells.clon)
    cx_glb = util_gather(cx)
    cy = np.rad2deg(domain.cells.clat)
    cy_glb = util_gather(cy)

    RHI_MAX_np_glb = util_gather(np.asarray(RHI_MAX))
    QI_MAX_np_glb = util_gather(np.asarray(QI_MAX))

    if rank == 0:
        # Create a figure with two subplots
        fig, axs = plt.subplots(1, 2, figsize=(10, 5), subplot_kw={'projection': cartopy.crs.PlateCarree()})
        axs[0].add_feature(cartopy.feature.BORDERS, edgecolor='gray')
        plot1 = axs[0].tricontourf(cx_glb, cy_glb, RHI_MAX_np_glb, transform=cartopy.crs.PlateCarree())
        axs[0].set_title('RHI_MAX')
        cbar1 = fig.colorbar(plot1, ax=axs[0], orientation='horizontal')

        axs[1].add_feature(cartopy.feature.BORDERS, edgecolor='gray')
        plot2 = axs[1].tricontourf(cx_glb, cy_glb, QI_MAX_np_glb, transform=cartopy.crs.PlateCarree())
        axs[1].set_title('QI_MAX')
        cbar2 = fig.colorbar(plot2, ax=axs[1], orientation='horizontal')
        # Adjust layout
        plt.tight_layout()
        # save the plot
        plt.savefig(f"/scratch/{user[0]}/{user}/icon_exercise_comin/P3_plotting_callback_function.png")
```

###  

### Run the ICON model

Now that you've finished writing your plugin, the final step is to run the ICON model again.   

This time, the plotting was conducted directly within the Python plugin. The obtained plot should look like <a href="reference/P3_comin_plugin/P3_plotting_callback_function.png"><code><code style="color:#2d4b9b">P3_plotting_callback_function.png</code></a>

<div class="alert alert-success">
  <b style="color:#2d4b9b;">Optional Exercise:</b> Finally, what would you have to change in order to plot at regular intervals?
    </div>

### Solution

  - You'd need to choose a suitable entry point, for example `EP_ATM_WRITE_OUTPUT_BEFORE`.
  - The simulation time would have to be queried so that the output does not occur too frequently: 
    - `descrdata_get_simulation_interval` returns simulation intervals: `exp_start`, `exp_stop`, `run_start`, `run_stop`
    - `current_get_datetime` 
    
 

---

<p style="height:2em;"/>

## Further Reading and Resources

- [1] ICON Tutorial, Ch. 9: https://www.dwd.de/DE/leistungen/nwv_icon_tutorial/nwv_icon_tutorial.html
- [2] Documentation on the ICON ComIn project website: https://icon-comin.gitlab-pages.dkrz.de/comin/ 
- [3] Python API description for ICON ComIn: https://icon-comin.gitlab-pages.dkrz.de/comin/d7/de6/md__2builds_2icon-comin_2comin_2doc_2comin__python__api.html

---

*Author info: Deutscher Wetterdienst (DWD) 2024 ::  comin@icon-model.org. For a full list of contributors, see CONTRIBUTING in the root directory. License info: see LICENSE file.*